## Library

In [1]:
import json
import random
from datasets import load_dataset, Dataset
from typing import List, Dict
import pandas as pd
from pathlib import Path
import gc, torch, time

In [2]:
from unsloth.chat_templates import get_chat_template

🦥
 
U
n
s
l
o
t
h
:
 
W
i
l
l
 
p
a
t
c
h
 
y
o
u
r
 
c
o
m
p
u
t
e
r
 
t
o
 
e
n
a
b
l
e
 
2
x
 
f
a
s
t
e
r
 
f
r
e
e
 
f
i
n
e
t
u
n
i
n
g
.

🦥
 
U
n
s
l
o
t
h
 
Z
o
o
 
w
i
l
l
 
n
o
w
 
p
a
t
c
h
 
e
v
e
r
y
t
h
i
n
g
 
t
o
 
m
a
k
e
 
t
r
a
i
n
i
n
g
 
f
a
s
t
e
r
!



In [3]:
from unsloth.chat_templates import standardize_sharegpt

### Load Model

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 30000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
#     "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

#     "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
#     "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
#     "unsloth/Llama-3.2-3B-bnb-4bit",
#     "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

=
=
(
(
=
=
=
=
)
)
=
=
 
 
U
n
s
l
o
t
h
 
2
0
2
4
.
1
1
.
1
0
:
 
F
a
s
t
 
L
l
a
m
a
 
p
a
t
c
h
i
n
g
.
 
T
r
a
n
s
f
o
r
m
e
r
s
:
4
.
4
6
.
3
.

 
 
 
\
\
 
 
 
/
|
 
 
 
 
G
P
U
:
 
N
V
I
D
I
A
 
G
e
F
o
r
c
e
 
R
T
X
 
4
0
9
0
.
 
M
a
x
 
m
e
m
o
r
y
:
 
2
3
.
5
3
5
 
G
B
.
 
P
l
a
t
f
o
r
m
:
 
L
i
n
u
x
.

O
^
O
/
 
\
_
/
 
\
 
 
 
 
T
o
r
c
h
:
 
2
.
5
.
1
.
 
C
U
D
A
:
 
8
.
9
.
 
C
U
D
A
 
T
o
o
l
k
i
t
:
 
1
2
.
1
.
 
T
r
i
t
o
n
:
 
3
.
1
.
0

\
 
 
 
 
 
 
 
 
/
 
 
 
 
B
f
l
o
a
t
1
6
 
=
 
T
R
U
E
.
 
F
A
 
[
X
f
o
r
m
e
r
s
 
=
 
0
.
0
.
2
8
.
p
o
s
t
3
.
 
F
A
2
 
=
 
F
a
l
s
e
]

 
"
-
_
_
_
_
-
"
 
 
 
 
 
F
r
e
e
 
A
p
a
c
h
e
 
l
i
c
e
n
s
e
:
 
h
t
t
p
:
/
/
g
i
t
h
u
b
.
c
o
m
/
u
n
s
l
o
t
h
a
i
/
u
n
s
l
o
t
h

U
n
s
l
o
t
h
:
 
F
a
s
t
 
d
o
w
n
l
o
a
d
i
n
g
 
i
s
 
e
n
a
b
l
e
d
 
-
 
i
g
n
o
r
e
 
d
o
w
n
l
o
a
d
i
n
g
 
b
a
r
s
 
w
h
i
c
h
 
a
r
e
 
r
e
d
 
c
o
l
o
r
e
d
!



In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head",], # Add for continual pretraining,
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

U
n
s
l
o
t
h
:
 
O
f
f
l
o
a
d
i
n
g
 
i
n
p
u
t
_
e
m
b
e
d
d
i
n
g
s
 
t
o
 
d
i
s
k
 
t
o
 
s
a
v
e
 
V
R
A
M



/
h
o
m
e
/
s
e
r
v
e
r
a
i
/
m
i
n
i
c
o
n
d
a
3
/
e
n
v
s
/
u
n
s
l
o
t
h
_
e
n
v
/
l
i
b
/
p
y
t
h
o
n
3
.
1
1
/
s
i
t
e
-
p
a
c
k
a
g
e
s
/
u
n
s
l
o
t
h
/
m
o
d
e
l
s
/
_
u
t
i
l
s
.
p
y
:
7
4
7
:
 
F
u
t
u
r
e
W
a
r
n
i
n
g
:
 
Y
o
u
 
a
r
e
 
u
s
i
n
g
 
`
t
o
r
c
h
.
l
o
a
d
`
 
w
i
t
h
 
`
w
e
i
g
h
t
s
_
o
n
l
y
=
F
a
l
s
e
`
 
(
t
h
e
 
c
u
r
r
e
n
t
 
d
e
f
a
u
l
t
 
v
a
l
u
e
)
,
 
w
h
i
c
h
 
u
s
e
s
 
t
h
e
 
d
e
f
a
u
l
t
 
p
i
c
k
l
e
 
m
o
d
u
l
e
 
i
m
p
l
i
c
i
t
l
y
.
 
I
t
 
i
s
 
p
o
s
s
i
b
l
e
 
t
o
 
c
o
n
s
t
r
u
c
t
 
m
a
l
i
c
i
o
u
s
 
p
i
c
k
l
e
 
d
a
t
a
 
w
h
i
c
h
 
w
i
l
l
 
e
x
e
c
u
t
e
 
a
r
b
i
t
r
a
r
y
 
c
o
d
e
 
d
u
r
i
n
g
 
u
n
p
i
c
k
l
i
n
g
 
(
S
e
e
 
h
t
t
p
s
:
/
/
g
i
t
h
u
b
.
c
o
m
/
p
y
t
o
r
c
h
/
p
y
t
o
r
c
h
/
b
l
o
b
/
m
a
i
n
/
S
E
C
U
R
I
T
Y
.
m
d
#
u
n
t
r
u
s
t
e
d
-
m
o
d
e
l
s
 
f
o
r
 
m
o
r
e
 
d
e
t
a
i
l
s
)
.
 
I
n
 
a
 
f
u
t
u
r
e
 
r
e
l
e
a
s
e
,
 
t
h
e
 
d
e
f
a
u
l
t
 
v
a
l
u
e
 
f
o
r
 
`
w
e
i
g
h
t
s
_


U
n
s
l
o
t
h
:
 
O
f
f
l
o
a
d
i
n
g
 
o
u
t
p
u
t
_
e
m
b
e
d
d
i
n
g
s
 
t
o
 
d
i
s
k
 
t
o
 
s
a
v
e
 
V
R
A
M



U
n
s
l
o
t
h
 
2
0
2
4
.
1
1
.
1
0
 
p
a
t
c
h
e
d
 
2
8
 
l
a
y
e
r
s
 
w
i
t
h
 
2
8
 
Q
K
V
 
l
a
y
e
r
s
,
 
2
8
 
O
 
l
a
y
e
r
s
 
a
n
d
 
2
8
 
M
L
P
 
l
a
y
e
r
s
.



U
n
s
l
o
t
h
:
 
T
r
a
i
n
i
n
g
 
e
m
b
e
d
_
t
o
k
e
n
s
 
i
n
 
m
i
x
e
d
 
p
r
e
c
i
s
i
o
n
 
t
o
 
s
a
v
e
 
V
R
A
M

U
n
s
l
o
t
h
:
 
T
r
a
i
n
i
n
g
 
l
m
_
h
e
a
d
 
i
n
 
m
i
x
e
d
 
p
r
e
c
i
s
i
o
n
 
t
o
 
s
a
v
e
 
V
R
A
M



## Data Preparation

### Functions

In [6]:
def free_mem():
  for _ in range(10):
        gc.collect()
        with torch.no_grad():
          torch.cuda.empty_cache()
        time.sleep(0.1)
      
def load_jsonl_dataset(file_path: str) -> Dataset:
    """
    Load a JSONL file into a Dataset object with 'conversations' feature.
    
    Args:
        file_path: Path to the JSONL file
        
    Returns:
        Dataset object with 'conversations' feature
    """
    # Read the JSONL file
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Parse each line as JSON and wrap in conversations format
            conversations = json.loads(line.strip())
            data.append({'conversations': conversations})
    
    # Convert to Dataset
    dataset = Dataset.from_list(data)
    
    return dataset

In [7]:
def xlsum_convert_to_sharegpt(example):
    """
    Convert a single XLSum example to ShareGPT format with a detailed prefix.
    """
    human_message = {
        "from": "human",
        "value": (
            "You are an expert summarization assistant trained to generate concise, clear, and accurate summaries of "
            "given texts. Your goal is to understand the core ideas of the provided article and create a summary and related keywords.\n\n"
            f"Provide a summary for the following article in its original language: \n"
            f"Title: {example['title']} \n"
            f"Content: {example['text']} \n"
            "Summary:"
        )
    }
    
    gpt_message = {
        "from": "gpt",
        "value": example['summary']
    }
    
    return [human_message, gpt_message]


def create_extended_conversation(examples: List[Dict], conversation_extension: int) -> List[Dict]:
    """
    Create an extended conversation by combining multiple examples.
    
    Args:
        examples: List of dataset examples to sample from
        conversation_extension: Number of examples to combine into one conversation
    
    Returns:
        List of messages forming a single extended conversation
    """
    # Randomly sample the specified number of examples
    selected_examples = random.sample(examples, min(conversation_extension, len(examples)))
    
    # Convert each example and combine their messages
    extended_conversation = []
    for example in selected_examples:
        conversation_pair = convert_to_sharegpt_format(example)
        extended_conversation.extend(conversation_pair)
    
    return extended_conversation

def process_dataset(dataset, conversation_extension: int = 1):
    """
    Process the dataset and convert to ShareGPT format with optional conversation extension.
    
    Args:
        dataset: The input dataset
        conversation_extension: Number of examples to combine into one conversation
    
    Returns:
        Dataset object with conversations column
    """
    processed_data = []
    
    # Process each split in the dataset
    for split in dataset.keys():
        print(f"Processing {split} split...")
        examples = list(dataset[split])
        
        # Calculate number of conversations needed
        num_conversations = len(examples) // conversation_extension
        
        # Create extended conversations
        for i in range(num_conversations):
            start_idx = i * conversation_extension
            end_idx = start_idx + conversation_extension
            conversation_examples = examples[start_idx:end_idx]
            
            # Create extended conversation
            if conversation_extension > 1:
                conversation = create_extended_conversation(conversation_examples, conversation_extension)
            else:
                conversation = xlsum_convert_to_sharegpt(conversation_examples[0])
            
            processed_data.append({"conversations": conversation})
    
    # Convert to Dataset object
    return Dataset.from_list(processed_data)

def save_sharegpt_format(dataset, output_path):
    """
    Save the converted data in JSONL format.
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in dataset:
            f.write(json.dumps(item['conversations'], ensure_ascii=False) + '\n')


### Download Data

#### Download Data - wikipedia-10k-id

In [4]:
# Define the dataset and subset
dataset_name = "indonesian-nlp/wikipedia-10k"
output_dir = Path("dataset/wikipedia-10k")

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Load the dataset
dataset = load_dataset(dataset_name, "wikipedia-id")

test-00000-of-00001.parquet:   0%|          | 0.00/30.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [6]:
print(dataset.column_names)
print(dataset['test'])

{
'
t
e
s
t
'
:
 
[
'
i
d
'
,
 
'
u
r
l
'
,
 
'
t
i
t
l
e
'
,
 
'
t
e
x
t
'
]
}

D
a
t
a
s
e
t
(
{

 
 
 
 
f
e
a
t
u
r
e
s
:
 
[
'
i
d
'
,
 
'
u
r
l
'
,
 
'
t
i
t
l
e
'
,
 
'
t
e
x
t
'
]
,

 
 
 
 
n
u
m
_
r
o
w
s
:
 
1
0
0
0
0

}
)



In [15]:
columns_to_keep = ['id', 'url', 'title', 'text']
standalone_dataset = dataset['test']
standalone_dataset = standalone_dataset.select_columns(columns_to_keep)

In [16]:
print(standalone_dataset.column_names)

[
'
i
d
'
,
 
'
u
r
l
'
,
 
'
t
i
t
l
e
'
,
 
'
t
e
x
t
'
]



In [17]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    standalone_dataset,
    merged_prompt = "Anda adalah sebuah asisten wikipedia. Jelaskan mengenai topik berikut: {title}",
    output_column_name = "text",
)

Merging columns:   0%|          | 0/10000 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
print(dataset[0])

In [21]:
output_path = f"{output_dir}/sharegpt_wikipedia-10k.jsonl"
save_sharegpt_format(dataset, output_path)

#### Download Data - alpaca-cleaned

In [55]:
# Define the dataset and subset
dataset_name = "yahma/alpaca-cleaned"
output_dir = Path("dataset/alpaca-cleaned")

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Load the dataset
dataset = load_dataset(dataset_name, split = "train")

In [56]:
print(dataset.column_names)
print(dataset[1])

[
'
o
u
t
p
u
t
'
,
 
'
i
n
p
u
t
'
,
 
'
i
n
s
t
r
u
c
t
i
o
n
'
]

{
'
o
u
t
p
u
t
'
:
 
'
T
h
e
 
t
h
r
e
e
 
p
r
i
m
a
r
y
 
c
o
l
o
r
s
 
a
r
e
 
r
e
d
,
 
b
l
u
e
,
 
a
n
d
 
y
e
l
l
o
w
.
 
T
h
e
s
e
 
c
o
l
o
r
s
 
a
r
e
 
c
a
l
l
e
d
 
p
r
i
m
a
r
y
 
b
e
c
a
u
s
e
 
t
h
e
y
 
c
a
n
n
o
t
 
b
e
 
c
r
e
a
t
e
d
 
b
y
 
m
i
x
i
n
g
 
o
t
h
e
r
 
c
o
l
o
r
s
 
a
n
d
 
a
l
l
 
o
t
h
e
r
 
c
o
l
o
r
s
 
c
a
n
 
b
e
 
m
a
d
e
 
b
y
 
c
o
m
b
i
n
i
n
g
 
t
h
e
m
 
i
n
 
v
a
r
i
o
u
s
 
p
r
o
p
o
r
t
i
o
n
s
.
 
I
n
 
t
h
e
 
a
d
d
i
t
i
v
e
 
c
o
l
o
r
 
s
y
s
t
e
m
,
 
u
s
e
d
 
f
o
r
 
l
i
g
h
t
,
 
t
h
e
 
p
r
i
m
a
r
y
 
c
o
l
o
r
s
 
a
r
e
 
r
e
d
,
 
g
r
e
e
n
,
 
a
n
d
 
b
l
u
e
 
(
R
G
B
)
.
'
,
 
'
i
n
p
u
t
'
:
 
'
'
,
 
'
i
n
s
t
r
u
c
t
i
o
n
'
:
 
'
W
h
a
t
 
a
r
e
 
t
h
e
 
t
h
r
e
e
 
p
r
i
m
a
r
y
 
c
o
l
o
r
s
?
'
}



In [57]:
print(dataset[0])

{
'
o
u
t
p
u
t
'
:
 
'
1
.
 
E
a
t
 
a
 
b
a
l
a
n
c
e
d
 
a
n
d
 
n
u
t
r
i
t
i
o
u
s
 
d
i
e
t
:
 
M
a
k
e
 
s
u
r
e
 
y
o
u
r
 
m
e
a
l
s
 
a
r
e
 
i
n
c
l
u
s
i
v
e
 
o
f
 
a
 
v
a
r
i
e
t
y
 
o
f
 
f
r
u
i
t
s
 
a
n
d
 
v
e
g
e
t
a
b
l
e
s
,
 
l
e
a
n
 
p
r
o
t
e
i
n
,
 
w
h
o
l
e
 
g
r
a
i
n
s
,
 
a
n
d
 
h
e
a
l
t
h
y
 
f
a
t
s
.
 
T
h
i
s
 
h
e
l
p
s
 
t
o
 
p
r
o
v
i
d
e
 
y
o
u
r
 
b
o
d
y
 
w
i
t
h
 
t
h
e
 
e
s
s
e
n
t
i
a
l
 
n
u
t
r
i
e
n
t
s
 
t
o
 
f
u
n
c
t
i
o
n
 
a
t
 
i
t
s
 
b
e
s
t
 
a
n
d
 
c
a
n
 
h
e
l
p
 
p
r
e
v
e
n
t
 
c
h
r
o
n
i
c
 
d
i
s
e
a
s
e
s
.
\
n
\
n
2
.
 
E
n
g
a
g
e
 
i
n
 
r
e
g
u
l
a
r
 
p
h
y
s
i
c
a
l
 
a
c
t
i
v
i
t
y
:
 
E
x
e
r
c
i
s
e
 
i
s
 
c
r
u
c
i
a
l
 
f
o
r
 
m
a
i
n
t
a
i
n
i
n
g
 
s
t
r
o
n
g
 
b
o
n
e
s
,
 
m
u
s
c
l
e
s
,
 
a
n
d
 
c
a
r
d
i
o
v
a
s
c
u
l
a
r
 
h
e
a
l
t
h
.
 
A
i
m
 
f
o
r
 
a
t
 
l
e
a
s
t
 
1
5
0
 
m
i
n
u
t
e
s
 
o
f
 
m
o
d
e
r
a
t
e
 
a
e
r
o
b
i
c
 
e
x
e
r
c
i
s
e
 
o
r
 
7
5
 
m
i
n
u
t
e
s
 
o
f
 
v


In [58]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 3, # Select more to handle longer conversations
)

Merging columns:   0%|          | 0/51760 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/51760 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/51760 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/51760 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/51760 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [59]:
print(dataset[0])

{
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
:
 
[
{
'
f
r
o
m
'
:
 
'
h
u
m
a
n
'
,
 
'
v
a
l
u
e
'
:
 
'
G
i
v
e
 
t
h
r
e
e
 
t
i
p
s
 
f
o
r
 
s
t
a
y
i
n
g
 
h
e
a
l
t
h
y
.
'
}
,
 
{
'
f
r
o
m
'
:
 
'
g
p
t
'
,
 
'
v
a
l
u
e
'
:
 
'
1
.
 
E
a
t
 
a
 
b
a
l
a
n
c
e
d
 
a
n
d
 
n
u
t
r
i
t
i
o
u
s
 
d
i
e
t
:
 
M
a
k
e
 
s
u
r
e
 
y
o
u
r
 
m
e
a
l
s
 
a
r
e
 
i
n
c
l
u
s
i
v
e
 
o
f
 
a
 
v
a
r
i
e
t
y
 
o
f
 
f
r
u
i
t
s
 
a
n
d
 
v
e
g
e
t
a
b
l
e
s
,
 
l
e
a
n
 
p
r
o
t
e
i
n
,
 
w
h
o
l
e
 
g
r
a
i
n
s
,
 
a
n
d
 
h
e
a
l
t
h
y
 
f
a
t
s
.
 
T
h
i
s
 
h
e
l
p
s
 
t
o
 
p
r
o
v
i
d
e
 
y
o
u
r
 
b
o
d
y
 
w
i
t
h
 
t
h
e
 
e
s
s
e
n
t
i
a
l
 
n
u
t
r
i
e
n
t
s
 
t
o
 
f
u
n
c
t
i
o
n
 
a
t
 
i
t
s
 
b
e
s
t
 
a
n
d
 
c
a
n
 
h
e
l
p
 
p
r
e
v
e
n
t
 
c
h
r
o
n
i
c
 
d
i
s
e
a
s
e
s
.
\
n
\
n
2
.
 
E
n
g
a
g
e
 
i
n
 
r
e
g
u
l
a
r
 
p
h
y
s
i
c
a
l
 
a
c
t
i
v
i
t
y
:
 
E
x
e
r
c
i
s
e
 
i
s
 
c
r
u
c
i
a
l
 
f
o
r
 
m
a
i
n
t
a
i
n
i
n
g
 
s
t
r
o
n
g
 
b
o
n
e
s
,
 
m
u
s
c
l
e
s
,
 
a
n


In [60]:
output_path = f"{output_dir}/sharegpt_alpaca-cleaned.jsonl"
save_sharegpt_format(dataset, output_path)

#### Download Data - alpaca-gpt4-indonesian

In [33]:
# Define the dataset and subset
dataset_name = "FreedomIntelligence/alpaca-gpt4-indonesian"
output_dir = Path("dataset/alpaca-gpt4-indonesian")

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Load the dataset
dataset = load_dataset(dataset_name, split = "train")

R
e
p
o
 
c
a
r
d
 
m
e
t
a
d
a
t
a
 
b
l
o
c
k
 
w
a
s
 
n
o
t
 
f
o
u
n
d
.
 
S
e
t
t
i
n
g
 
C
a
r
d
D
a
t
a
 
t
o
 
e
m
p
t
y
.



In [30]:
print(dataset.column_names)
print(dataset[0])

[
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
,
 
'
i
d
'
]

{
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
:
 
[
{
'
f
r
o
m
'
:
 
'
h
u
m
a
n
'
,
 
'
v
a
l
u
e
'
:
 
'
S
a
r
a
n
l
a
h
 
s
l
o
g
a
n
 
u
n
t
u
k
 
k
a
m
p
a
n
y
e
 
d
a
u
r
 
u
l
a
n
g
\
n
'
}
,
 
{
'
f
r
o
m
'
:
 
'
g
p
t
'
,
 
'
v
a
l
u
e
'
:
 
'
1
.
 
"
K
u
r
a
n
g
i
,
 
g
u
n
a
k
a
n
 
k
e
m
b
a
l
i
,
 
d
a
u
r
 
u
l
a
n
g
:
 
B
e
r
s
a
m
a
 
u
n
t
u
k
 
m
a
s
a
 
d
e
p
a
n
 
y
a
n
g
 
l
e
b
i
h
 
h
i
j
a
u
.
"
\
n
2
.
 
"
D
a
u
r
 
u
l
a
n
g
l
a
h
 
h
a
r
i
 
i
n
i
,
 
u
n
t
u
k
 
m
a
s
a
 
d
e
p
a
n
 
y
a
n
g
 
l
e
b
i
h
 
b
a
i
k
.
"
\
n
3
.
 
"
U
b
a
h
 
s
a
m
p
a
h
m
u
 
m
e
n
j
a
d
i
 
h
a
r
t
a
 
k
a
r
u
n
 
-
 
D
a
u
r
 
u
l
a
n
g
!
"
\
n
4
.
 
"
D
a
u
r
 
u
l
a
n
g
 
u
n
t
u
k
 
s
i
k
l
u
s
 
k
e
h
i
d
u
p
a
n
.
"
\
n
5
.
 
"
S
i
m
p
a
n
 
s
u
m
b
e
r
 
d
a
y
a
,
 
d
a
u
r
 
u
l
a
n
g
 
l
e
b
i
h
 
b
a
n
y
a
k
.
"
'
}
]
,
 
'
i
d
'
:
 
'
2
3
7
1
2
'
}



In [31]:
output_path = f"{output_dir}/sharegpt_alpaca-gpt4-indonesian.jsonl"
save_sharegpt_format(dataset, output_path)

#### Download Data - finetome100k

In [23]:
# Define the dataset and subset
dataset_name = "mlabonne/FineTome-100k"
output_dir = Path("dataset/FineTome-100k")

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Load the dataset
dataset = load_dataset(dataset_name, split = "train")

In [24]:
print(dataset.column_names)
print(dataset[0])

[
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
,
 
'
s
o
u
r
c
e
'
,
 
'
s
c
o
r
e
'
]

{
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
:
 
[
{
'
f
r
o
m
'
:
 
'
h
u
m
a
n
'
,
 
'
v
a
l
u
e
'
:
 
'
E
x
p
l
a
i
n
 
w
h
a
t
 
b
o
o
l
e
a
n
 
o
p
e
r
a
t
o
r
s
 
a
r
e
,
 
w
h
a
t
 
t
h
e
y
 
d
o
,
 
a
n
d
 
p
r
o
v
i
d
e
 
e
x
a
m
p
l
e
s
 
o
f
 
h
o
w
 
t
h
e
y
 
c
a
n
 
b
e
 
u
s
e
d
 
i
n
 
p
r
o
g
r
a
m
m
i
n
g
.
 
A
d
d
i
t
i
o
n
a
l
l
y
,
 
d
e
s
c
r
i
b
e
 
t
h
e
 
c
o
n
c
e
p
t
 
o
f
 
o
p
e
r
a
t
o
r
 
p
r
e
c
e
d
e
n
c
e
 
a
n
d
 
p
r
o
v
i
d
e
 
e
x
a
m
p
l
e
s
 
o
f
 
h
o
w
 
i
t
 
a
f
f
e
c
t
s
 
t
h
e
 
e
v
a
l
u
a
t
i
o
n
 
o
f
 
b
o
o
l
e
a
n
 
e
x
p
r
e
s
s
i
o
n
s
.
 
D
i
s
c
u
s
s
 
t
h
e
 
d
i
f
f
e
r
e
n
c
e
 
b
e
t
w
e
e
n
 
s
h
o
r
t
-
c
i
r
c
u
i
t
 
e
v
a
l
u
a
t
i
o
n
 
a
n
d
 
n
o
r
m
a
l
 
e
v
a
l
u
a
t
i
o
n
 
i
n
 
b
o
o
l
e
a
n
 
e
x
p
r
e
s
s
i
o
n
s
 
a
n
d
 
d
e
m
o
n
s
t
r
a
t
e
 
t
h
e
i
r
 
u
s
a
g
e
 
i
n
 
c
o
d
e
.
 
\
n
\
n
F
u
r
t
h
e
r
m
o
r
e
,
 
a
d
d
 
t
h
e
 
r
e
q
u
i
r
e

In [26]:
print(dataset.column_names)

[
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
,
 
'
s
o
u
r
c
e
'
,
 
'
s
c
o
r
e
'
]



In [27]:
output_path = f"{output_dir}/sharegpt_FineTome-100k.jsonl"
save_sharegpt_format(dataset, output_path)

#### Download Data - xlsum-english

In [2]:
# Define the dataset and subset
dataset_name = "csebuetnlp/xlsum"
language_subset = "english"
output_dir = Path("dataset/xlsum-english")

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Load the dataset
dataset = load_dataset(dataset_name, language_subset, split = "test")

0000.parquet:   0%|          | 0.00/315M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/264M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/306522 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11535 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11535 [00:00<?, ? examples/s]

In [4]:
dataset
dataset.column_names

['id', 'url', 'title', 'summary', 'text']

In [5]:
dataset 

Dataset({
    features: ['id', 'url', 'title', 'summary', 'text'],
    num_rows: 11535
})

In [7]:
def xlsum_en_convert_to_sharegpt(dataset):
    """
    Convert the dataset into ShareGPT format.

    Parameters:
    dataset (Dataset): The dataset to be converted.

    Returns:
    Dataset: A new dataset in ShareGPT format.
    """
    sharegpt_data = []

    for example in dataset:
        # Format the input message
        human_message = (
            f"Provide a summary for the following article in its original language: \nTitle: {example['title']} \nContent: {example['text']} \nSummary:"
        )
        
        # Format the output message
        gpt_message = (
            f"{example['summary']}"
        )
        
        # Create the ShareGPT format
        conversation = [
            {"from": "human", "value": human_message},
            {"from": "gpt", "value": gpt_message}
        ]
        sharegpt_data.append(conversation)
    
    # Convert to Dataset format
    return sharegpt_data

In [8]:
dataset = xlsum_en_convert_to_sharegpt(dataset)

In [10]:
dataset = Dataset.from_dict({"conversations": dataset})

In [13]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 11535
})

In [14]:
output_path = f"{output_dir}/sharegpt_xlsum_english.jsonl"
save_sharegpt_format(dataset, output_path)

#### Download Data - xlsum-indonesian

##### Download 

In [2]:
# Define the dataset and subset
dataset_name = "csebuetnlp/xlsum"
language_subset = "indonesian"
output_dir = Path("dataset/xlsum-indonesian")

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Load the dataset
dataset = load_dataset(dataset_name, language_subset, split = "train")

In [3]:
print(dataset.column_names)

[
'
i
d
'
,
 
'
u
r
l
'
,
 
'
t
i
t
l
e
'
,
 
'
s
u
m
m
a
r
y
'
,
 
'
t
e
x
t
'
]



In [ ]:
print(dataset['train'][0])

Save to CSV

In [ ]:
for split in ["train", "test", "validation"]:
    df = pd.DataFrame(dataset[split])  # Convert to Pandas DataFrame
    output_file = output_dir / f"xlsum_indonesian_{split}.csv"
    df.to_csv(output_file, index=False, encoding="utf-8")
    print(f"Saved {split} split to {output_file}")

##### Load Dataset

In [17]:
train_path = "dataset/xlsum-indonesian/xlsum_indonesian_train.csv"
test_path = "dataset/xlsum-indonesian/xlsum_indonesian_test.csv"
validation_path = "dataset/xlsum-indonesian/xlsum_indonesian_validation.csv"

dataset = load_dataset("csv", data_files={
    "train": train_path,
    "test": test_path,
    "validation": validation_path
})

In [18]:
dataset['train'][0]

{'id': 'media-49647079',
 'url': 'https://www.bbc.com/indonesia/media-49647079',
 'title': 'Gajah mengamuk saat upacara keagamaan, 17 orang terluka',
 'summary': 'Seekor gajah mendadak mengamuk saat prosesi tahunan agama Buddha di Kolombo, Sri Lanka, sehingga membuat peserta upacara tunggang-langgang. Setidaknya 17 orang terluka.',
 'text': 'Dilaporkan dua orang terluka cukup serius, sementara sisanya sudah diperbolehkan pulang setelah mendapatkan perawatan. Video gajah yang mengamuk itu kontan viral di media sosial. Si gajah berlari tak tentu arah, menabrak serta menginjak sebagian peserta upacara. Simak juga: Belum diketahui penyebab gajah tersebut tiba-tiba mengamuk. Diduga gajah itu kaget oleh sesuatu di antara para peserta dan pengunjung. Media setempat melaporkan gajah lain yang juga mengamuk di prosesi berbeda. Gajah hias merupakan daya tarik tersendiri dalam upacara keagamaan di Sri Lanka. Bagi warga Sri Lanka, memiliki gajah adalah simbol status. Beberapa kuil di Sri Lanka jug

##### Convert to sharegpt

###### Unsloth 

In [ ]:
##this code doesnt work, out of index bug

from unsloth import to_sharegpt
train_dataset = to_sharegpt(
    dataset['validation'],
    merged_prompt= \
        "[[Provide a summary for the following article in its original language:]]"\
        "[[\nTitle: {title}]]"\
        "[[\nContent: {text}]]"\
        "[[\nSummarize below:]]",
    conversation_extension=2,  # Randomly combines conversations 
    output_column_name="summary",  # Use the "summary" column as the target
)


###### Manual

In [19]:
# Set conversation extension (e.g., 3 for combining 3 examples into one conversation)
conversation_extension = 1

# Convert to ShareGPT format with conversation extension
dataset = process_dataset(dataset, conversation_extension)

P
r
o
c
e
s
s
i
n
g
 
t
r
a
i
n
 
s
p
l
i
t
.
.
.

P
r
o
c
e
s
s
i
n
g
 
t
e
s
t
 
s
p
l
i
t
.
.
.

P
r
o
c
e
s
s
i
n
g
 
v
a
l
i
d
a
t
i
o
n
 
s
p
l
i
t
.
.
.



In [20]:
# Print a sample conversation
print("\nSample conversation:")
print(json.dumps(dataset[0]['conversations'], ensure_ascii=False, indent=2))


S
a
m
p
l
e
 
c
o
n
v
e
r
s
a
t
i
o
n
:

[

 
 
{

 
 
 
 
"
f
r
o
m
"
:
 
"
h
u
m
a
n
"
,

 
 
 
 
"
v
a
l
u
e
"
:
 
"
Y
o
u
 
a
r
e
 
a
n
 
e
x
p
e
r
t
 
s
u
m
m
a
r
i
z
a
t
i
o
n
 
a
s
s
i
s
t
a
n
t
 
t
r
a
i
n
e
d
 
t
o
 
g
e
n
e
r
a
t
e
 
c
o
n
c
i
s
e
,
 
c
l
e
a
r
,
 
a
n
d
 
a
c
c
u
r
a
t
e
 
s
u
m
m
a
r
i
e
s
 
o
f
 
g
i
v
e
n
 
t
e
x
t
s
.
 
Y
o
u
r
 
g
o
a
l
 
i
s
 
t
o
 
u
n
d
e
r
s
t
a
n
d
 
t
h
e
 
c
o
r
e
 
i
d
e
a
s
 
o
f
 
t
h
e
 
p
r
o
v
i
d
e
d
 
a
r
t
i
c
l
e
 
a
n
d
 
c
r
e
a
t
e
 
a
 
s
u
m
m
a
r
y
 
a
n
d
 
r
e
l
a
t
e
d
 
k
e
y
w
o
r
d
s
.
\
n
\
n
P
r
o
v
i
d
e
 
a
 
s
u
m
m
a
r
y
 
f
o
r
 
t
h
e
 
f
o
l
l
o
w
i
n
g
 
a
r
t
i
c
l
e
 
i
n
 
i
t
s
 
o
r
i
g
i
n
a
l
 
l
a
n
g
u
a
g
e
:
 
\
n
T
i
t
l
e
:
 
G
a
j
a
h
 
m
e
n
g
a
m
u
k
 
s
a
a
t
 
u
p
a
c
a
r
a
 
k
e
a
g
a
m
a
a
n
,
 
1
7
 
o
r
a
n
g
 
t
e
r
l
u
k
a
 
\
n
C
o
n
t
e
n
t
:
 
D
i
l
a
p
o
r
k
a
n
 
d
u
a
 
o
r
a
n
g
 
t
e
r
l
u
k
a
 
c
u
k
u
p
 
s
e
r
i
u
s
,
 
s
e
m
e
n
t
a
r
a
 
s
i
s
a
n
y
a
 
s
u
d
a

In [21]:
# Print dataset info
print("\nDataset info:")
print(f"Number of conversations: {len(dataset)}")
print(f"Features: {dataset.features}")
print(f"Datatype: {type(dataset)}")


D
a
t
a
s
e
t
 
i
n
f
o
:

N
u
m
b
e
r
 
o
f
 
c
o
n
v
e
r
s
a
t
i
o
n
s
:
 
4
7
8
0
2

F
e
a
t
u
r
e
s
:
 
{
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
:
 
[
{
'
f
r
o
m
'
:
 
V
a
l
u
e
(
d
t
y
p
e
=
'
s
t
r
i
n
g
'
,
 
i
d
=
N
o
n
e
)
,
 
'
v
a
l
u
e
'
:
 
V
a
l
u
e
(
d
t
y
p
e
=
'
s
t
r
i
n
g
'
,
 
i
d
=
N
o
n
e
)
}
]
}

D
a
t
a
t
y
p
e
:
 
<
c
l
a
s
s
 
'
d
a
t
a
s
e
t
s
.
a
r
r
o
w
_
d
a
t
a
s
e
t
.
D
a
t
a
s
e
t
'
>



In [22]:
# Save the converted data
output_path = "dataset/xlsum-indonesian/sharegpt_xlsum_indonesian.jsonl"
save_sharegpt_format(dataset, output_path)
print(f"Converted data saved to {output_path}")

C
o
n
v
e
r
t
e
d
 
d
a
t
a
 
s
a
v
e
d
 
t
o
 
d
a
t
a
s
e
t
/
x
l
s
u
m
-
i
n
d
o
n
e
s
i
a
n
/
s
h
a
r
e
g
p
t
_
x
l
s
u
m
_
i
n
d
o
n
e
s
i
a
n
.
j
s
o
n
l



#### Download Data - scientific_lay_summarisation-plos-norm

In [3]:
# Define the dataset and subset
dataset_name = "pszemraj/scientific_lay_summarisation-plos-norm"
output_dir = Path("dataset/scientific_lay_summarisation-plos-norm")

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Load the dataset
dataset = load_dataset(dataset_name, split = "train")

In [4]:
print(dataset)
print(dataset[0])

D
a
t
a
s
e
t
(
{

 
 
 
 
f
e
a
t
u
r
e
s
:
 
[
'
a
r
t
i
c
l
e
'
,
 
'
s
u
m
m
a
r
y
'
,
 
'
s
e
c
t
i
o
n
_
h
e
a
d
i
n
g
s
'
,
 
'
k
e
y
w
o
r
d
s
'
,
 
'
y
e
a
r
'
,
 
'
t
i
t
l
e
'
,
 
'
a
r
t
i
c
l
e
_
l
e
n
g
t
h
'
,
 
'
s
u
m
m
a
r
y
_
l
e
n
g
t
h
'
]
,

 
 
 
 
n
u
m
_
r
o
w
s
:
 
2
4
7
7
3

}
)

{
'
a
r
t
i
c
l
e
'
:
 
"
K
i
d
n
e
y
 
f
u
n
c
t
i
o
n
 
d
e
p
e
n
d
s
 
o
n
 
t
h
e
 
n
e
p
h
r
o
n
,
 
w
h
i
c
h
 
c
o
m
p
r
i
s
e
s
 
a
 
b
l
o
o
d
 
f
i
l
t
e
r
,
 
a
 
t
u
b
u
l
e
 
t
h
a
t
 
i
s
 
s
u
b
d
i
v
i
d
e
d
 
i
n
t
o
 
f
u
n
c
t
i
o
n
a
l
l
y
 
d
i
s
t
i
n
c
t
 
s
e
g
m
e
n
t
s
,
 
a
n
d
 
a
 
c
o
l
l
e
c
t
i
n
g
 
d
u
c
t
.
 
H
o
w
 
t
h
e
s
e
 
r
e
g
i
o
n
s
 
a
r
i
s
e
 
d
u
r
i
n
g
 
d
e
v
e
l
o
p
m
e
n
t
 
i
s
 
p
o
o
r
l
y
 
u
n
d
e
r
s
t
o
o
d
.
 
T
h
e
 
z
e
b
r
a
f
i
s
h
 
p
r
o
n
e
p
h
r
o
s
 
c
o
n
s
i
s
t
s
 
o
f
 
t
w
o
 
l
i
n
e
a
r
 
n
e
p
h
r
o
n
s
 
t
h
a
t
 
d
e
v
e
l
o
p
 
f
r
o
m
 
t
h
e
 
i
n
t
e
r
m
e
d
i
a
t
e
 
m
e
s
o
d
e
r
m
 
a
l
o
n
g
 
t


In [5]:
def scientific_convert_to_sharegpt(dataset):
    """
    Convert the dataset into ShareGPT format.

    Parameters:
    dataset (Dataset): The dataset to be converted.

    Returns:
    Dataset: A new dataset in ShareGPT format.
    """
    sharegpt_data = []

    for example in dataset:
        # Format the input message
        human_message = (
            "You are an expert summarization assistant trained to generate concise, clear, and accurate summaries of "
            "given texts. Your goal is to understand the core ideas of the provided article and create a summary and related keywords.\n\n"
            f"Title: {example['title']}\n\n"
            f"Content: {example['article']}\n\n"
            "Summary:\n"
        )
        
        # Format the output message
        gpt_message = (
            f"{example['summary']}\n\n"
            f"keywords: {example['keywords']}"
        )
        
        # Create the ShareGPT format
        conversation = [
            {"from": "human", "value": human_message},
            {"from": "gpt", "value": gpt_message}
        ]
        sharegpt_data.append(conversation)
    
    # Convert to Dataset format
    return sharegpt_data

In [6]:
sharegpt_dataset = scientific_convert_to_sharegpt(dataset)
print(sharegpt_dataset[0])

[
{
'
f
r
o
m
'
:
 
'
h
u
m
a
n
'
,
 
'
v
a
l
u
e
'
:
 
"
Y
o
u
 
a
r
e
 
a
n
 
e
x
p
e
r
t
 
s
u
m
m
a
r
i
z
a
t
i
o
n
 
a
s
s
i
s
t
a
n
t
 
t
r
a
i
n
e
d
 
t
o
 
g
e
n
e
r
a
t
e
 
c
o
n
c
i
s
e
,
 
c
l
e
a
r
,
 
a
n
d
 
a
c
c
u
r
a
t
e
 
s
u
m
m
a
r
i
e
s
 
o
f
 
g
i
v
e
n
 
t
e
x
t
s
.
 
Y
o
u
r
 
g
o
a
l
 
i
s
 
t
o
 
u
n
d
e
r
s
t
a
n
d
 
t
h
e
 
c
o
r
e
 
i
d
e
a
s
 
o
f
 
t
h
e
 
p
r
o
v
i
d
e
d
 
a
r
t
i
c
l
e
 
a
n
d
 
c
r
e
a
t
e
 
a
 
s
u
m
m
a
r
y
 
a
n
d
 
r
e
l
a
t
e
d
 
k
e
y
w
o
r
d
s
.
\
n
\
n
T
i
t
l
e
:
 
T
h
e
 
c
d
x
 
G
e
n
e
s
 
a
n
d
 
R
e
t
i
n
o
i
c
 
A
c
i
d
 
C
o
n
t
r
o
l
 
t
h
e
 
P
o
s
i
t
i
o
n
i
n
g
 
a
n
d
 
S
e
g
m
e
n
t
a
t
i
o
n
 
o
f
 
t
h
e
 
Z
e
b
r
a
f
i
s
h
 
P
r
o
n
e
p
h
r
o
s
\
n
\
n
C
o
n
t
e
n
t
:
 
K
i
d
n
e
y
 
f
u
n
c
t
i
o
n
 
d
e
p
e
n
d
s
 
o
n
 
t
h
e
 
n
e
p
h
r
o
n
,
 
w
h
i
c
h
 
c
o
m
p
r
i
s
e
s
 
a
 
b
l
o
o
d
 
f
i
l
t
e
r
,
 
a
 
t
u
b
u
l
e
 
t
h
a
t
 
i
s
 
s
u
b
d
i
v
i
d
e
d
 
i
n
t
o
 
f
u
n
c
t
i
o
n
a
l
l
y
 
d
i
s
t


In [7]:
dataset = Dataset.from_dict({"conversations": sharegpt_dataset})

# Check the first example
print(dataset[0])

{
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
:
 
[
{
'
f
r
o
m
'
:
 
'
h
u
m
a
n
'
,
 
'
v
a
l
u
e
'
:
 
"
Y
o
u
 
a
r
e
 
a
n
 
e
x
p
e
r
t
 
s
u
m
m
a
r
i
z
a
t
i
o
n
 
a
s
s
i
s
t
a
n
t
 
t
r
a
i
n
e
d
 
t
o
 
g
e
n
e
r
a
t
e
 
c
o
n
c
i
s
e
,
 
c
l
e
a
r
,
 
a
n
d
 
a
c
c
u
r
a
t
e
 
s
u
m
m
a
r
i
e
s
 
o
f
 
g
i
v
e
n
 
t
e
x
t
s
.
 
Y
o
u
r
 
g
o
a
l
 
i
s
 
t
o
 
u
n
d
e
r
s
t
a
n
d
 
t
h
e
 
c
o
r
e
 
i
d
e
a
s
 
o
f
 
t
h
e
 
p
r
o
v
i
d
e
d
 
a
r
t
i
c
l
e
 
a
n
d
 
c
r
e
a
t
e
 
a
 
s
u
m
m
a
r
y
 
a
n
d
 
r
e
l
a
t
e
d
 
k
e
y
w
o
r
d
s
.
\
n
\
n
T
i
t
l
e
:
 
T
h
e
 
c
d
x
 
G
e
n
e
s
 
a
n
d
 
R
e
t
i
n
o
i
c
 
A
c
i
d
 
C
o
n
t
r
o
l
 
t
h
e
 
P
o
s
i
t
i
o
n
i
n
g
 
a
n
d
 
S
e
g
m
e
n
t
a
t
i
o
n
 
o
f
 
t
h
e
 
Z
e
b
r
a
f
i
s
h
 
P
r
o
n
e
p
h
r
o
s
\
n
\
n
C
o
n
t
e
n
t
:
 
K
i
d
n
e
y
 
f
u
n
c
t
i
o
n
 
d
e
p
e
n
d
s
 
o
n
 
t
h
e
 
n
e
p
h
r
o
n
,
 
w
h
i
c
h
 
c
o
m
p
r
i
s
e
s
 
a
 
b
l
o
o
d
 
f
i
l
t
e
r
,
 
a
 
t
u
b
u
l
e
 
t
h
a
t
 
i
s
 
s
u
b
d
i
v
i
d
e
d
 
i
n
t
o


In [8]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 24773
})

In [9]:
output_path = f"{output_dir}/sharegpt_scientific_lay_summarisation-plos-norm.jsonl"
save_sharegpt_format(dataset, output_path)

#### Download Data - govreport-summarization-8192

In [22]:
# Define the dataset and subset
dataset_name = "pszemraj/govreport-summarization-8192"
output_dir = Path("dataset/govreport-summarization-8192")

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Load the dataset
dataset = load_dataset(dataset_name, split = "train")

README.md:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/234M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/55.7M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/13.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8200 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/440 [00:00<?, ? examples/s]

In [23]:
print(dataset.column_names)
print(dataset[0])

[
'
r
e
p
o
r
t
'
,
 
'
s
u
m
m
a
r
y
'
,
 
'
i
n
p
u
t
_
t
o
k
e
n
_
l
e
n
'
,
 
'
s
u
m
m
a
r
y
_
t
o
k
e
n
_
l
e
n
'
]

{
'
r
e
p
o
r
t
'
:
 
"
T
h
e
r
e
 
a
r
e
 
s
o
m
e
 
s
i
m
i
l
a
r
i
t
i
e
s
 
i
n
 
h
o
w
 
M
e
d
i
c
a
r
e
 
p
a
y
s
 
A
S
C
s
 
a
n
d
 
h
o
s
p
i
t
a
l
 
o
u
t
p
a
t
i
e
n
t
 
d
e
p
a
r
t
m
e
n
t
s
 
f
o
r
 
t
h
e
 
p
r
o
c
e
d
u
r
e
s
 
t
h
e
y
 
p
e
r
f
o
r
m
.
 
H
o
w
e
v
e
r
,
 
t
h
e
 
m
e
t
h
o
d
s
 
u
s
e
d
 
b
y
 
C
M
S
 
t
o
 
c
a
l
c
u
l
a
t
e
 
t
h
e
 
p
a
y
m
e
n
t
 
r
a
t
e
s
 
i
n
 
e
a
c
h
 
s
y
s
t
e
m
,
 
a
s
 
w
e
l
l
 
a
s
 
t
h
e
 
m
e
c
h
a
n
i
s
m
s
 
u
s
e
d
 
t
o
 
r
e
v
i
s
e
 
t
h
e
 
M
e
d
i
c
a
r
e
 
p
a
y
m
e
n
t
 
r
a
t
e
s
,
 
d
i
f
f
e
r
.
 
I
n
 
1
9
8
0
,
 
l
e
g
i
s
l
a
t
i
o
n
 
w
a
s
 
e
n
a
c
t
e
d
 
t
h
a
t
 
e
n
a
b
l
e
d
 
A
S
C
s
 
t
o
 
b
i
l
l
 
M
e
d
i
c
a
r
e
 
f
o
r
 
c
e
r
t
a
i
n
 
s
u
r
g
i
c
a
l
 
p
r
o
c
e
d
u
r
e
s
 
p
r
o
v
i
d
e
d
 
t
o
 
M
e
d
i
c
a
r
e
 
b
e
n
e
f
i
c
i
a
r
i
e
s
.
 
U
n
d
e
r
 
t
h
e
 
A

In [24]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "You are an expert summarization assistant trained to generate concise, clear, and accurate summaries of given texts. Your goal is to understand the core ideas of the provided article and create a summary. [[\nHere is the article:\n{report}]]",
    output_column_name = "summary",
    # conversation_extension = 3, # Select more to handle longer conversations
)

Merging columns:   0%|          | 0/8200 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/8200 [00:00<?, ? examples/s]

In [25]:
print(dataset.column_names)
print(dataset[0])

[
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
]

{
'
c
o
n
v
e
r
s
a
t
i
o
n
s
'
:
 
[
{
'
f
r
o
m
'
:
 
'
h
u
m
a
n
'
,
 
'
v
a
l
u
e
'
:
 
'
Y
o
u
 
a
r
e
 
a
n
 
e
x
p
e
r
t
 
s
u
m
m
a
r
i
z
a
t
i
o
n
 
a
s
s
i
s
t
a
n
t
 
t
r
a
i
n
e
d
 
t
o
 
g
e
n
e
r
a
t
e
 
c
o
n
c
i
s
e
,
 
c
l
e
a
r
,
 
a
n
d
 
a
c
c
u
r
a
t
e
 
s
u
m
m
a
r
i
e
s
 
o
f
 
g
i
v
e
n
 
t
e
x
t
s
.
 
Y
o
u
r
 
g
o
a
l
 
i
s
 
t
o
 
u
n
d
e
r
s
t
a
n
d
 
t
h
e
 
c
o
r
e
 
i
d
e
a
s
 
o
f
 
t
h
e
 
p
r
o
v
i
d
e
d
 
a
r
t
i
c
l
e
 
a
n
d
 
c
r
e
a
t
e
 
a
 
s
u
m
m
a
r
y
.
 
\
n
H
e
r
e
 
i
s
 
t
h
e
 
a
r
t
i
c
l
e
:
\
n
(
"
T
h
e
r
e
 
a
r
e
 
s
o
m
e
 
s
i
m
i
l
a
r
i
t
i
e
s
 
i
n
 
h
o
w
 
M
e
d
i
c
a
r
e
 
p
a
y
s
 
A
S
C
s
 
a
n
d
 
h
o
s
p
i
t
a
l
 
o
u
t
p
a
t
i
e
n
t
 
d
e
p
a
r
t
m
e
n
t
s
 
f
o
r
 
t
h
e
 
p
r
o
c
e
d
u
r
e
s
 
t
h
e
y
 
p
e
r
f
o
r
m
.
 
H
o
w
e
v
e
r
,
 
t
h
e
 
m
e
t
h
o
d
s
 
u
s
e
d
 
b
y
 
C
M
S
 
t
o
 
c
a
l
c
u
l
a
t
e
 
t
h
e
 
p
a
y
m
e
n
t
 
r
a
t
e
s
 
i
n
 
e
a
c
h
 
s
y
s
t
e
m
,
 
a
s

In [26]:
output_path = f"{output_dir}/sharegpt_govreport-summarization-8192.jsonl"
save_sharegpt_format(dataset, output_path)

## Dataset Preparation

### Combine Dataset

In [8]:
# Selects a given number of rows from a dataset
# Will be off due to rounding errors***
def subset(dataset : Dataset, count : int) -> Dataset:
  divisor = int(len(dataset) / count)
  new_dataset = dataset[::divisor]
  while len(new_dataset['conversations']) > count:
    new_dataset['conversations'].pop()
  return Dataset.from_dict(new_dataset)

# Way slower but more precise
def subset_slow_exact(dataset : Dataset, count : int) -> Dataset:
  divisor = len(dataset) / count
  new_dataset = {'conversations':[]}
  i = 0
  while i < len(dataset) - 1:
    new_dataset['conversations'].append(dataset['conversations'][int(i)])
    i += divisor
  return Dataset.from_dict(new_dataset)

In [8]:
data_path = Path("dataset")

In [10]:
finetune_dataset = data_path / 'sharegpt_finetune.jsonl'

In [11]:
finetune_dataset = load_jsonl_dataset(str(finetune_dataset))

In [ ]:
# alpaca_cleaned = data_path / 'alpaca-cleaned/sharegpt_alpaca-cleaned.jsonl'
# alpaca_id = data_path / 'alpaca-gpt4-indonesian/sharegpt_alpaca-gpt4-indonesian.jsonl'
# finetome_100k = data_path / 'FineTome-100k/sharegpt_FineTome-100k.jsonl'
# wikipedia_10k = data_path / 'wikipedia-10k/sharegpt_wikipedia-10k.jsonl'
xlsum_en = data_path / 'xlsum-english/sharegpt_xlsum_english.jsonl'
xlsum_id = data_path / 'xlsum-indonesian/sharegpt_xlsum_indonesian.jsonl'
govreport = data_path / 'govreport-summarization-8192/sharegpt_govreport-summarization-8192.jsonl'
scientific_sum = data_path / 'scientific_lay_summarisation-plos-norm/sharegpt_scientific_3k.jsonl'
        
# Load the dataset for each path
# alpaca_cleaned = load_jsonl_dataset(str(alpaca_cleaned))
# alpaca_id = load_jsonl_dataset(str(alpaca_id))
# finetome_100k = load_jsonl_dataset(str(finetome_100k))
# wikipedia_10k = load_jsonl_dataset(str(wikipedia_10k))
xlsum_en = load_jsonl_dataset(str(xlsum_en))
xlsum_id = load_jsonl_dataset(str(xlsum_id))
govreport = load_jsonl_dataset(str(govreport))
scientific_sum = load_jsonl_dataset(str(scientific_sum))

# Print the number of rows in each dataset as a quick verification
# print(f"Alpaca Cleaned: {len(alpaca_cleaned)} examples")
# print(f"Alpaca Indonesian: {len(alpaca_id)} examples")
# print(f"FineTome 100k: {len(finetome_100k)} examples")
# print(f"Wikipedia 10k: {len(wikipedia_10k)} examples")
print(f"XLSum English: {len(xlsum_en)} examples")
print(f"XLSum Indonesian: {len(xlsum_id)} examples")
print(f"GovReport: {len(govreport)} examples")
print(f"Scientific Summary: {len(scientific_sum)} examples")

In [11]:
from datasets import concatenate_datasets

# Combine datasets together
# pretrain_dataset = concatenate_datasets([alpaca_cleaned, alpaca_id, finetome_100k, wikipedia_10k])
finetune_dataset = concatenate_datasets([xlsum_id, xlsum_en, govreport, scientific_sum])
# finetune_dataset = concatenate_datasets([xlsum_id, govreport, scientific_sum])

# Shuffle dataset (optional)
# pretrain_dataset = pretrain_dataset.shuffle(seed=0)
# finetune_dataset = finetune_dataset.shuffle(seed=0)
# scientific_sum = scientific_sum.shuffle(seed=0)
# scientific_sum = subset_slow_exact(scientific_sum, 3000)

In [16]:
# pretrain_dataset

Dataset({
    features: ['conversations'],
    num_rows: 211729
})

In [12]:
finetune_dataset

Dataset({
    features: ['conversations'],
    num_rows: 70535
})

In [20]:
def count_tokens_in_conversation(conversation, tokenizer):
    """
    Count tokens in a single conversation.
    """
    total_tokens = 0
    
    # Concatenate all messages in the conversation
    full_text = ""
    for message in conversation:
        # Add message format
        full_text += f"{message['from']}: {message['value']}\n"
    
    # Count tokens
    tokens = tokenizer(full_text, return_tensors="pt", truncation=False)
    return len(tokens.input_ids[0])

def analyze_dataset_tokens(dataset, tokenizer):
    """
    Analyze token counts across the entire dataset.
    """
    token_counts = []
    
    # Process each conversation
    for idx, item in enumerate(dataset):
        if idx % 1000 == 0:  # Progress indicator
            print(f"Processing conversation {idx}/{len(dataset)}")
            
        tokens = count_tokens_in_conversation(item['conversations'], tokenizer)
        token_counts.append(tokens)
    
    # Calculate statistics
    total_tokens = sum(token_counts)
    avg_tokens = total_tokens / len(token_counts)
    max_tokens = max(token_counts)
    min_tokens = min(token_counts)
    
    # Find conversations exceeding token limit
    over_limit = sum(1 for count in token_counts if count > max_seq_length)
    
    return {
        "total_tokens": total_tokens,
        "average_tokens": avg_tokens,
        "max_tokens": max_tokens,
        "min_tokens": min_tokens,
        "total_conversations": len(token_counts),
        "conversations_over_limit": over_limit,
        "percent_over_limit": (over_limit / len(token_counts)) * 100
    }

In [19]:
# # Use the functions
# stats = analyze_dataset_tokens(pretrain_dataset, tokenizer)

In [20]:
# # Print results
# print("\nPretrain Dataset Token Statistics:")
# print(f"Total Tokens: {stats['total_tokens']:,}")
# print(f"Average Tokens per Conversation: {stats['average_tokens']:.2f}")
# print(f"Max Tokens in a Conversation: {stats['max_tokens']:,}")
# print(f"Min Tokens in a Conversation: {stats['min_tokens']:,}")
# print(f"Total Conversations: {stats['total_conversations']:,}")
# print(f"Conversations Over {max_seq_length:,} tokens: {stats['conversations_over_limit']:,} ({stats['percent_over_limit']:.2f}%)")

In [21]:
stats = analyze_dataset_tokens(scientific_sum, tokenizer)

P
r
o
c
e
s
s
i
n
g
 
c
o
n
v
e
r
s
a
t
i
o
n
 
0
/
3
0
0
0

P
r
o
c
e
s
s
i
n
g
 
c
o
n
v
e
r
s
a
t
i
o
n
 
1
0
0
0
/
3
0
0
0

P
r
o
c
e
s
s
i
n
g
 
c
o
n
v
e
r
s
a
t
i
o
n
 
2
0
0
0
/
3
0
0
0



In [22]:
# Print results
print("\nFinetune Dataset Token Statistics:")
print(f"Total Tokens: {stats['total_tokens']:,}")
print(f"Average Tokens per Conversation: {stats['average_tokens']:.2f}")
print(f"Max Tokens in a Conversation: {stats['max_tokens']:,}")
print(f"Min Tokens in a Conversation: {stats['min_tokens']:,}")
print(f"Total Conversations: {stats['total_conversations']:,}")
print(f"Conversations Over {max_seq_length:,} tokens: {stats['conversations_over_limit']:,} ({stats['percent_over_limit']:.2f}%)")


F
i
n
e
t
u
n
e
 
D
a
t
a
s
e
t
 
T
o
k
e
n
 
S
t
a
t
i
s
t
i
c
s
:

T
o
t
a
l
 
T
o
k
e
n
s
:
 
2
7
,
6
4
3
,
2
1
9

A
v
e
r
a
g
e
 
T
o
k
e
n
s
 
p
e
r
 
C
o
n
v
e
r
s
a
t
i
o
n
:
 
9
2
1
4
.
4
1

M
a
x
 
T
o
k
e
n
s
 
i
n
 
a
 
C
o
n
v
e
r
s
a
t
i
o
n
:
 
2
7
,
3
6
5

M
i
n
 
T
o
k
e
n
s
 
i
n
 
a
 
C
o
n
v
e
r
s
a
t
i
o
n
:
 
1
,
2
3
6

T
o
t
a
l
 
C
o
n
v
e
r
s
a
t
i
o
n
s
:
 
3
,
0
0
0

C
o
n
v
e
r
s
a
t
i
o
n
s
 
O
v
e
r
 
4
0
,
0
0
0
 
t
o
k
e
n
s
:
 
0
 
(
0
.
0
0
%
)



### Standardize

In [14]:
from unsloth import standardize_sharegpt
# pretrain_dataset = standardize_sharegpt(pretrain_dataset)
finetune_dataset = standardize_sharegpt(finetune_dataset)

Standardizing format:   0%|          | 0/70537 [00:00<?, ? examples/s]

In [15]:
# Free the variables
del xlsum_id
del govreport
del scientific_sum

# Optionally, force garbage collection
import gc
gc.collect()


39

In [ ]:
# print("\nSample conversation (pretrain):")
# print(json.dumps(pretrain_dataset[1]['conversations'], ensure_ascii=False, indent=2))

In [13]:
print("\nSample conversation (finetune):")
print(json.dumps(finetune_dataset[1]['conversations'], ensure_ascii=False, indent=2))


S
a
m
p
l
e
 
c
o
n
v
e
r
s
a
t
i
o
n
 
(
f
i
n
e
t
u
n
e
)
:

[

 
 
{

 
 
 
 
"
c
o
n
t
e
n
t
"
:
 
"
Y
o
u
 
a
r
e
 
a
n
 
e
x
p
e
r
t
 
s
u
m
m
a
r
i
z
a
t
i
o
n
 
a
s
s
i
s
t
a
n
t
 
t
r
a
i
n
e
d
 
t
o
 
g
e
n
e
r
a
t
e
 
c
o
n
c
i
s
e
,
 
c
l
e
a
r
,
 
a
n
d
 
a
c
c
u
r
a
t
e
 
s
u
m
m
a
r
i
e
s
 
o
f
 
g
i
v
e
n
 
t
e
x
t
s
.
 
Y
o
u
r
 
g
o
a
l
 
i
s
 
t
o
 
u
n
d
e
r
s
t
a
n
d
 
t
h
e
 
c
o
r
e
 
i
d
e
a
s
 
o
f
 
t
h
e
 
p
r
o
v
i
d
e
d
 
a
r
t
i
c
l
e
 
a
n
d
 
c
r
e
a
t
e
 
a
 
s
u
m
m
a
r
y
 
a
n
d
 
r
e
l
a
t
e
d
 
k
e
y
w
o
r
d
s
.
\
n
\
n
P
r
o
v
i
d
e
 
a
 
s
u
m
m
a
r
y
 
f
o
r
 
t
h
e
 
f
o
l
l
o
w
i
n
g
 
a
r
t
i
c
l
e
 
i
n
 
i
t
s
 
o
r
i
g
i
n
a
l
 
l
a
n
g
u
a
g
e
:
 
\
n
T
i
t
l
e
:
 
A
p
a
 
a
l
a
s
a
n
 
p
e
m
e
r
i
n
t
a
h
 
p
a
n
g
k
a
s
 
1
4
 
d
a
l
a
m
 
d
a
f
t
a
r
 
P
r
o
y
e
k
 
S
t
r
a
t
e
g
i
s
 
N
a
s
i
o
n
a
l
 
?
 
\
n
C
o
n
t
e
n
t
:
 
P
r
o
y
e
k
 
M
R
T
 
S
u
d
i
r
m
a
n
-
 
L
e
b
a
k
 
B
u
l
u
s
 
t
e
n
g
a
h
 
d
i
k
e
r
j
a
k
a
n
,
 
n
a


## Training

### Chat Template

In [14]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

# def formatting_prompts_func(examples):
#     convos = examples["conversations"]
#     texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
#     return { "text" : texts, }
# pass

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = []
    
    for convo in convos:
        try:
            # Skip invalid conversations
            if not convo or not isinstance(convo, list):
                texts.append("")
                continue
                
            # Validate and clean conversation messages
            valid_convo = [
                {
                    "role": msg["role"],
                    "content": msg["content"] or ""  # Convert None to empty string
                }
                for msg in convo
                if isinstance(msg, dict) 
                and msg.get("role") 
                and msg.get("content") is not None
            ]
            
            if not valid_convo:
                texts.append("")
                continue
                
            text = tokenizer.apply_chat_template(
                valid_convo,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)
        except Exception as e:
            print(f"Error processing conversation: {e}")
            texts.append("")
            
    return {"text": texts}

def clean_dataset(dataset):
    """
    Clean the dataset by removing conversations with None values
    and validating conversation structure
    """
    def is_valid_conversation(conv):
        if not conv or not isinstance(conv, list):
            return False
        return all(
            isinstance(msg, dict) 
            and isinstance(msg.get('role'), str) 
            and isinstance(msg.get('content'), str)
            for msg in conv
        )
    
    # Filter valid conversations
    filtered_dataset = dataset.filter(
        lambda x: is_valid_conversation(x['conversations']),
        num_proc=4  # Adjust based on your CPU cores
    )
    
    print(f"Original dataset size: {len(dataset)}")
    print(f"Cleaned dataset size: {len(filtered_dataset)}")
    
    return filtered_dataset

In [27]:
# pretrain_dataset = pretrain_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/211729 [00:00<?, ? examples/s]

In [15]:
finetune_dataset = finetune_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/70535 [00:00<?, ? examples/s]

In [ ]:
# pretrain_dataset[5]["conversations"]

In [ ]:
# pretrain_dataset[5]["text"]

In [16]:
finetune_dataset[37269]["conversations"]

[{'content': "You are an expert summarization assistant trained to generate concise, clear, and accurate summaries of given texts. Your goal is to understand the core ideas of the provided article and create a summary and related keywords.\n\nProvide a summary for the following article in its original language: \nTitle: Israel 'setujui' empat permukiman baru di Tepi Barat \nContent: Masyarakat internasional menilai semua permukiman di Tepi Barat tidak sah. Dokumen yang diajukan ke Mahkamah Agung menunjukkan pemerintah telah memulai proses persetujuan empat permukiman, yang saat ini berstatus tidak sah. Salah satu di antaranya, Givat Assaf di dekat Ramallah, tadinya akan dihancurkan. Seorang pejabat Palestina menggambarkan langkah ini sebagai tamparan keras terhadap upaya perdamaian di kawasan. Pada 2005 laporan pemerintah menyebutkan warga di Givat Assaf, Givat HaRoeh, Maaleh Rehavam, dan Mitzpe Lachish agar meninggalkan kawasan tersebut. Sikap masyarakat internasional Tidak lama kemud

In [17]:
finetune_dataset[37269]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are an expert summarization assistant trained to generate concise, clear, and accurate summaries of given texts. Your goal is to understand the core ideas of the provided article and create a summary and related keywords.\n\nProvide a summary for the following article in its original language: \nTitle: Israel 'setujui' empat permukiman baru di Tepi Barat \nContent: Masyarakat internasional menilai semua permukiman di Tepi Barat tidak sah. Dokumen yang diajukan ke Mahkamah Agung menunjukkan pemerintah telah memulai proses persetujuan empat permukiman, yang saat ini berstatus tidak sah. Salah satu di antaranya, Givat Assaf di dekat Ramallah, tadinya akan dihancurkan. Seorang pejabat Palestina menggambarkan langkah ini sebagai tamparan keras terhadap upaya perdamaian di kawasan. Pada 2005 laporan pemeri

In [18]:
# Clean the dataset before training
finetune_dataset = clean_dataset(finetune_dataset)

Filter (num_proc=4):   0%|          | 0/70535 [00:00<?, ? examples/s]

O
r
i
g
i
n
a
l
 
d
a
t
a
s
e
t
 
s
i
z
e
:
 
7
0
5
3
5

C
l
e
a
n
e
d
 
d
a
t
a
s
e
t
 
s
i
z
e
:
 
7
0
5
3
5



In [29]:
output_path = f"dataset/sharegpt_finetune.jsonl"
save_sharegpt_format(finetune_dataset, output_path)

### Continued Pretraining

In [19]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = finetune_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        # warmup_steps = 10,
        warmup_ratio = 0.01,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps =None,
        # learning_rate = 2e-4,
        learning_rate = 2e-4,
        # embedding_learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        save_strategy = "steps",
        save_steps = 100,
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/70535 [00:00<?, ? examples/s]

In [ ]:
# from trl import SFTTrainer
# from transformers import TrainingArguments, DataCollatorForSeq2Seq
# from unsloth import is_bfloat16_supported
# from unsloth import UnslothTrainer, UnslothTrainingArguments

# trainer = UnslothTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     # train_dataset = pretrain_dataset,
#     train_dataset = finetune_dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
#     dataset_num_proc = 2,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = UnslothTrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 8,
#         # warmup_steps = 10,
#         warmup_ratio = 0.05,
#         num_train_epochs = 2, # Set this for 1 full training run.
#         # max_steps =None,
#         # learning_rate = 2e-4,
#         learning_rate = 5e-5,
#         embedding_learning_rate = 1e-5,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#         save_strategy = "steps",
#         save_steps = 100,
#         report_to = "none", # Use this for WandB etc
#     ),
# )

Map (num_proc=2):   0%|          | 0/80775 [00:00<?, ? examples/s]

In [23]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/70535 [00:00<?, ? examples/s]

In [24]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are an expert summarization assistant trained to generate concise, clear, and accurate summaries of given texts. Your goal is to understand the core ideas of the provided article and create a summary and related keywords.\n\nProvide a summary for the following article in its original language: \nTitle: Jadi Kepala BNPT, Tito Karnavian diyakini akan buat perubahan besar \nContent: Tito Karnavian kini menjabat kepala BNPT, menggantikan Saud Usman Nasution. Dengan dilantiknya Tito sebagai kepala BNPT, jabatan Kapolda Metro Jaya akan diserahkan ke Inspektur Jenderal Polisi Moechgiyarto, yang sebelumnya menjabat Kapolda Jawa Barat. Tito juga akan mendapatkan kenaikan pangkat menjadi Komisaris Jenderal atau bintang tiga. Penunjukan Tito, yang pernah menjadi Kepala Detasemen Khusus 88 pada November 2009 hin

In [25]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

#### Show memory stats

In [26]:
#@title Show current memory stats
free_mem()
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

G
P
U
 
=
 
N
V
I
D
I
A
 
G
e
F
o
r
c
e
 
R
T
X
 
4
0
9
0
.
 
M
a
x
 
m
e
m
o
r
y
 
=
 
2
3
.
5
3
5
 
G
B
.

4
.
7
4
 
G
B
 
o
f
 
m
e
m
o
r
y
 
r
e
s
e
r
v
e
d
.



In [ ]:
 model.config.use_cache = False

#### Train

In [ ]:
# use this to train from scratch
# trainer_stats = trainer.train()

# use this to train from checkpoint
trainer_stats = trainer.train(resume_from_checkpoint = True)


/
h
o
m
e
/
s
e
r
v
e
r
a
i
/
m
i
n
i
c
o
n
d
a
3
/
e
n
v
s
/
u
n
s
l
o
t
h
_
e
n
v
/
l
i
b
/
p
y
t
h
o
n
3
.
1
1
/
s
i
t
e
-
p
a
c
k
a
g
e
s
/
t
r
a
n
s
f
o
r
m
e
r
s
/
t
r
a
i
n
e
r
.
p
y
:
3
3
5
4
:
 
F
u
t
u
r
e
W
a
r
n
i
n
g
:
 
Y
o
u
 
a
r
e
 
u
s
i
n
g
 
`
t
o
r
c
h
.
l
o
a
d
`
 
w
i
t
h
 
`
w
e
i
g
h
t
s
_
o
n
l
y
=
F
a
l
s
e
`
 
(
t
h
e
 
c
u
r
r
e
n
t
 
d
e
f
a
u
l
t
 
v
a
l
u
e
)
,
 
w
h
i
c
h
 
u
s
e
s
 
t
h
e
 
d
e
f
a
u
l
t
 
p
i
c
k
l
e
 
m
o
d
u
l
e
 
i
m
p
l
i
c
i
t
l
y
.
 
I
t
 
i
s
 
p
o
s
s
i
b
l
e
 
t
o
 
c
o
n
s
t
r
u
c
t
 
m
a
l
i
c
i
o
u
s
 
p
i
c
k
l
e
 
d
a
t
a
 
w
h
i
c
h
 
w
i
l
l
 
e
x
e
c
u
t
e
 
a
r
b
i
t
r
a
r
y
 
c
o
d
e
 
d
u
r
i
n
g
 
u
n
p
i
c
k
l
i
n
g
 
(
S
e
e
 
h
t
t
p
s
:
/
/
g
i
t
h
u
b
.
c
o
m
/
p
y
t
o
r
c
h
/
p
y
t
o
r
c
h
/
b
l
o
b
/
m
a
i
n
/
S
E
C
U
R
I
T
Y
.
m
d
#
u
n
t
r
u
s
t
e
d
-
m
o
d
e
l
s
 
f
o
r
 
m
o
r
e
 
d
e
t
a
i
l
s
)
.
 
I
n
 
a
 
f
u
t
u
r
e
 
r
e
l
e
a
s
e
,
 
t
h
e
 
d
e
f
a
u
l
t
 
v
a
l
u
e
 
f
o
r
 
`
w
e
i
g
h
t
s
_


Step,Training Loss


#### Final Memory anf Time Stats

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")